# DATA STRUCT BLOCK
Đoạn code trên thực hiện các thao tác phân tích dữ liệu điểm thi của các thí sinh theo từng khối và xác định khối thi tốt nhất cho từng thí sinh, dựa trên các môn thi của họ.

In [ ]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
import os

cleaned_data_api = 'https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download'

def fetch_csv_from_api(api_url):
    cache_file = 'cleaned_data_cache.csv'
    cache_timeout = timedelta(hours=24)
    
    if os.path.exists(cache_file):
        modified_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - modified_time < cache_timeout:
            print(f"Đang tải dữ liệu từ cache {cache_file}...")
            return pd.read_csv(cache_file)
    
    print(f"Đang tải dữ liệu từ API {api_url}...")
    response = requests.get(api_url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv(cache_file, index=False)
        return df
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Tải dữ liệu từ API
try:
    data_df = fetch_csv_from_api(cleaned_data_api)
    print("Đã tải dữ liệu thành công từ API")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu từ API: {e}")
    print("Không thể tải dữ liệu. Vui lòng kiểm tra kết nối internet và thử lại.")
    exit()

# Define block calculations
data_df['Khoi_A'] = data_df[['Toan', 'Ly', 'Hoa']].mean(axis=1)
data_df['Khoi_B'] = data_df[['Toan', 'Sinh', 'Hoa']].mean(axis=1)
data_df['Khoi_C'] = data_df[['Van', 'Lich su', 'Dia ly']].mean(axis=1)
data_df['Khoi_D'] = data_df[['Toan', 'Van', 'Ngoai ngu']].mean(axis=1)

# Check if any block has exactly two scores (indicating "Thí Sinh Thi Lại")
data_df['Thi_Lai'] = (
    (data_df[['Toan', 'Ly', 'Hoa']].notnull().sum(axis=1) == 2) |
    (data_df[['Toan', 'Sinh', 'Hoa']].notnull().sum(axis=1) == 2) |
    (data_df[['Van', 'Lich su', 'Dia ly']].notnull().sum(axis=1) == 2) |
    (data_df[['Toan', 'Van', 'Ngoai ngu']].notnull().sum(axis=1) == 2)
)

# Determine the best block for students not marked as "Thi Lai"
data_df['Best_Khoi'] = data_df[['Khoi_A', 'Khoi_B', 'Khoi_C', 'Khoi_D']].idxmax(axis=1)
data_df['Best_Khoi'] = data_df.apply(lambda row: "Thí Sinh Thi Lại" if row['Thi_Lai'] else row['Best_Khoi'], axis=1)

# Select relevant columns for output
block_analysis_final_df = data_df[['SBD', 'Best_Khoi', 'TenTinh']]

# Export the final result to CSV
block_analysis_final_df.to_csv('Block_Analysis_Final.csv', index=False)
